# Some fun with &pi; in Julia

## &pi; in Julia
*(Simon Byrne)*

Like most technical languages, Julia provides a variable constant for &pi;. However Julia's handling is a bit special.

In [ ]:
pi

It can also be accessed via the unicode symbol (you can get it at the REPL or in a notebook via the TeX completion `\pi` followed by a tab)

In [ ]:
π

You'll notice that it doesn't print like an ordinary floating point number: that's because it isn't one.

In [ ]:
typeof(pi)

&pi; and a few other irrational constants are instead stored as special `Irrational` values, rather than being rounded to `Float64`. These act like ordinary numeric values, except that they can are converted automatically to any floating point type without any intermediate rounding:

In [ ]:
1 + pi # integers are promoted to Float64 by default

In [ ]:
Float32(1) + pi # Float32

This is particularly useful for use with arbitrary-precision `BigFloat`s, as &pi; can be evaluated to full precision (rather than be truncated to `Float64` and converted back).

In [ ]:
BigFloat(1) + pi # 256 bits by default

If &pi; were stored as a `Float64`, we would instead get

In [ ]:
BigFloat(1) + Float64(pi)

In fact `BigFloat` (which uses the [MPFR](http://www.mpfr.org) library) will compute &pi; on demand to the current precision, which is set via `setprecision`. This provides an easy way to get its digits: 

In [ ]:
# to 1024 bits
setprecision(BigFloat, 1024) do 
    BigFloat(pi)
end

The last few digits may be incorrect due to the conversion from binary to decimal.

## &pi; via inline assembly instructions
*(Simon Byrne)*

Julia provides a very low-level `llvmcall` interface, which allows the user to directly write [LLVM intermediate representation](http://llvm.org/docs/LangRef.html), including the use of inline assembly. The following snippet calls the `fldpi` instruction ("**f**loating point **l**oa**d** **pi**") which loads the constant &pi; onto the floating point register stack (this works only on x86 and x86_64 architectures)

In [ ]:
function asm_pi()
    Base.llvmcall(
    """ %pi = call double asm "fldpi", "={st}"()
        ret double %pi""", 
    Float64, Tuple{})
end

In [ ]:
asm_pi()

We can look at the actual resulting code that is generated:

In [ ]:
@code_native asm_pi()

If you're wondering what the rest of these instructions are doing:

1. the `pushq` and `movq` adds to the [call stack frame](https://en.wikipedia.org/wiki/Call_stack).
2. `fldpi` pushes &pi; to the x87 floating point register stack
  - x87 is the older legacy floating point instruction set dating back to the original [Intel 8087 coprocessor](https://en.wikipedia.org/wiki/Intel_8087).
3. `fstpl` and `movsd` moves the value to the SSE floating point register `xmm0`
  - Julia, like most modern software, uses the newer SSE instruction set for its floating point operations. This also allows us to take advantage of things like [SIMD operations](https://en.wikipedia.org/wiki/SIMD).
4. `popq` and `retq` pops the call stack frame.

## &pi; using a Taylor series expansions
*([Luis Benet](https://github.com/lbenet))*

### Madhava's formula

One of the standard trigonmetric identities is
$$ \tan\left( \frac{\pi}{6} \right) = \frac{1}{\sqrt{3}}. $$

Therefore, by taking the Taylor expansion of $6 \arctan(x)$ around 0 we may obtain the value of $\pi$, by evaluating it at $1/\sqrt{3}$, a value which is within the radius of convergence.

In [ ]:
using TaylorSeries

We obtain the Taylor series of order 37th, using `BigFloat`s:

In [ ]:
series1 = 6atan( Taylor1(BigFloat, 37) )
convert(Taylor1{Rational{BigInt}},series1)

Note that the series above has only odd powers, so we will be using in this case 18 coefficients.

Evaluating that expression in $1/\sqrt{3}$ we get

In [ ]:
pi_approx1 = evaluate(series1, 1/sqrt(big(3)))

Then, the 37th order Taylor expansion yields a value which differs from $\pi$ in:

In [ ]:
abs(pi - pi_approx1)

To obtain more accurate results, we may simply increase the order of the expansion:

In [ ]:
series2 = 6atan( Taylor1(BigFloat,99) ) # 49 coefficients of the series
pi_approx2 = evaluate(series2, 1/sqrt(BigInt(3)))

In [ ]:
abs(pi - pi_approx2)

This formulation is one of the [*Madhava* or *Gregory–Leibniz series*](https://en.wikipedia.org/wiki/Madhava_series#Another_formula_for_the_circumference_of_a_circle):

\begin{equation}
\pi = 6 \sum_{n=0}^{\infty} (-1)^n \frac{(1/\sqrt{3})^{2n+1}}{2n+1}.
\end{equation}

### Machin's approach

Following the same idea, [John Machin](https://en.wikipedia.org/wiki/John_Machin#Formula) derived an algorithm which converges much faster, using the identity

\begin{equation}
\frac{\pi}{4} = 4 \arctan\left(\frac{1}{5}\right) - \arctan\left(\frac{1}{239}\right).
\end{equation}

Following what we did above, using again a 37th Taylor expansion:

In [ ]:
ser = atan( Taylor1(BigFloat, 37) )
pi_approx3 = 4*( 4*evaluate(ser, 1/big(5)) - evaluate(ser, 1/big(239)) )

In [ ]:
abs(pi - pi_approx3)

### References


[Wikipedia article](https://en.wikipedia.org/wiki/Pi#Infinite_series) about Pi (section infinite series).